# Package List

In [9]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

# Data Load

In [10]:
df = pd.read_csv("../data/서울시대여공구찾기정보.csv", encoding='cp949')

## Preprocessing

In [11]:
del_col = df.isnull().sum()[df.isnull().sum()/df.shape[0]>0.95].index.tolist()
df = df[[x for x in df.columns if x not in del_col]]
df['생성일시'] = pd.to_datetime(df['생성일시'], format='%Y%m%d%H%M%S')
del df['위치(위도,경도)']
del df['요금']

In [12]:
df['요금(과금기준 + 요금)'].fillna('0원', inplace=True)
def split_period_and_price(x):
    if '1일' in x:
        return pd.Series(['1일', x.replace('1일', '').strip()])
    elif '1시간' in x:
        return pd.Series(['1시간', x.replace('1시간', '').strip()])
    else:
        return pd.Series(['없음', x.strip()])

df[['과금기준', '요금']] = df['요금(과금기준 + 요금)'].apply(split_period_and_price)
del df['요금(과금기준 + 요금)']

# Location Info 

In [13]:
import requests

def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '강남역'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


강남역 위도/경도: (37.4999348, 127.0269678)


In [ ]:
api_key =''
openai.api_key = api_key

In [21]:
from openai import OpenAI

In [23]:
client = OpenAI(api_key=api_key)

In [28]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response

In [29]:
output = invoke('배가 고픈데 생선을 먹고싶어. 6월에 먹을 수 있는 제철 생선 있어?')

TypeError: Responses.create() got an unexpected keyword argument 'messages'

In [20]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야돼
    """
    response = openai.ChatCompletion.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "당신은 완벽한 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    result_text = response['choices'][0]['message']['content']

    try:
        result_dict = json.loads(result_text)
        with open("clustered_equipment.json", "w", encoding="utf-8") as f:
            json.dump(result_dict, f, ensure_ascii=False, indent=2)
        print("✅ 클러스터링 결과 저장 완료: clustered_equipment.json")
        return result_dict
    except json.JSONDecodeError:
        print("⚠️ JSON 형식 오류. 출력 확인 필요:")
        print(result_text)
        return None
    

In [19]:
question = '나 너무 배고파'
result = invoke(question)

# 만약 JSON 결과를 바로 보고 싶으면
import pandas as pd

# if result:
#     df_result = pd.DataFrame(result)
#     display(df_result)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [25]:
prompt = f"""
    너는 도구가 어디있는지 찾아주는 도우미야. 
    사용자가 필요한 도구와 현재 위치를 알려주거야.
    너는 {df}를 참고해서 **사용자가 필요로 하는 도구**와 **현재 위치**에서 가장 가까이에있는 대여소를 찾아주면 돼.

    """